In [2]:
import tvm
import pandas as pd
%pylab
df=pd.read_pickle('data/result1.pkl',compression='gzip')
df['targets']=df['targets'].apply(str)

Using matplotlib backend: TkAgg
Populating the interactive namespace from numpy and matplotlib


In [3]:
df['targets']

0      llvm -mcpu=core-avx2
1      llvm -mcpu=core-avx2
2      llvm -mcpu=core-avx2
3      llvm -mcpu=core-avx2
4      llvm -mcpu=core-avx2
               ...         
595     cuda -model=unknown
596     cuda -model=unknown
597     cuda -model=unknown
598     cuda -model=unknown
599     cuda -model=unknown
Name: targets, Length: 600, dtype: object

In [4]:
_dataset=[]
num=0
for idx,value in df[df['targets']=='cuda -model=unknown'].groupby(['weights','strides','paddings']):
#     print(idx)
    value['idx']=num
    num+=1
    _dataset.append(value[['idx','vectors','cost']].values)
    
    



/home/jaehun/workspace/pytorch-maml/env/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [5]:
dataset=np.concatenate(_dataset)

In [6]:
dataset.shape

(360, 3)

In [7]:
dataset[:,1][0].shape


(482,)

In [8]:
from torchmeta.datasets.helpers import omniglot
from torchmeta.utils.data import BatchMetaDataLoader

dataset = omniglot("data", ways=5, shots=5, test_shots=15, meta_train=True, download=True)
dataloader = BatchMetaDataLoader(dataset, batch_size=16, num_workers=4)
_train=[]
for batch in dataloader:
    train_inputs, train_targets = batch["train"]
    print('Train inputs shape: {0}'.format(train_inputs.shape))    # (16, 25, 1, 28, 28)
    print('Train targets shape: {0}'.format(train_targets.shape))  # (16, 25)
    _train.extend(train_targets.numpy().tolist())
    test_inputs, test_targets = batch["test"]
    print('Test inputs shape: {0}'.format(test_inputs.shape))      # (16, 75, 1, 28, 28)
    print('Test targets shape: {0}'.format(test_targets.shape))    # (16, 75)
    break

Train inputs shape: torch.Size([16, 25, 1, 28, 28])
Train targets shape: torch.Size([16, 25])
Test inputs shape: torch.Size([16, 75, 1, 28, 28])
Test targets shape: torch.Size([16, 75])


In [9]:
dataset = omniglot("data", ways=5, shots=5, test_shots=5, meta_train=True, download=True)


In [10]:
dataset = omniglot("data", ways=5, shots=5, test_shots=15, meta_train=True, download=True)


In [11]:
dataset

In [12]:
len(_train)

16

In [13]:
len(set(list(map(str,_train))))

16

In [1]:
import pandas as pd
df=pd.read_pickle('data/result1.pkl',compression='gzip')
df['targets']=df['targets'].apply(str)

In [2]:
df

,names,knobs,weights,strides,paddings,indexs,targets,networks,vectors,cost
0,topi_x86_conv2d_NCHWc,tile_ic-tile_oc-tile_ow-unroll_kw,"(64, 3, 7, 7)","(2, 2)","(3, 3, 3, 3)",217,llvm -mcpu=core-avx2,resnet-18,"[0.0, 0.0, 0.0, 1.0, 0.0, 2.0, 0.0, 0.0, 0.0, ...",Traceback (most recent call last):\n [bt] (3)...
1,topi_x86_conv2d_NCHWc,tile_ic-tile_oc-tile_ow-unroll_kw,"(64, 3, 7, 7)","(2, 2)","(3, 3, 3, 3)",238,llvm -mcpu=core-avx2,resnet-18,"[0.0, 0.0, 0.0, 1.0, 0.0, 2.0, 0.0, 3.0, 0.0, ...",Traceback (most recent call last):\n [bt] (3)...
2,topi_x86_conv2d_NCHWc,tile_ic-tile_oc-tile_ow-unroll_kw,"(64, 3, 7, 7)","(2, 2)","(3, 3, 3, 3)",14,llvm -mcpu=core-avx2,resnet-18,"[0.0, 0.0, 27.814382553100586, 0.0, 27.8143825...",Traceback (most recent call last):\n [bt] (3)...
3,topi_x86_conv2d_NCHWc,tile_ic-tile_oc-tile_ow-unroll_kw,"(64, 3, 7, 7)","(2, 2)","(3, 3, 3, 3)",34,llvm -mcpu=core-avx2,resnet-18,"[0.0, 0.0, 0.0, 1.0, 0.0, 2.0, 0.0, 0.0, 0.0, ...",Traceback (most recent call last):\n [bt] (3)...
4,topi_x86_conv2d_NCHWc,tile_ic-tile_oc-tile_ow-unroll_kw,"(64, 3, 7, 7)","(2, 2)","(3, 3, 3, 3)",43,llvm -mcpu=core-avx2,resnet-18,"[0.0, 0.0, 0.0, 1.0, 0.0, 2.0, 26.007026672363...",Traceback (most recent call last):\n [bt] (3)...
...,...,...,...,...,...,...,...,...,...,...
595,topi_nn_conv2d,tile_f-tile_y-tile_x-tile_rc-tile_ry-tile_rx-a...,"(512, 512, 3, 3)","(1, 1)","(1, 1, 1, 1)",226910,cuda -model=unknown,resnet-18,"[0.0, 0.0, 0.0, 1.0, 0.0, 2.0, 0.0, 0.19264507...",0.0209023
596,topi_nn_conv2d,tile_f-tile_y-tile_x-tile_rc-tile_ry-tile_rx-a...,"(512, 512, 3, 3)","(1, 1)","(1, 1, 1, 1)",249525,cuda -model=unknown,resnet-18,"[0.0, 0.0, 0.0, 1.0, 0.0, 2.0, 25.977279663085...",0.0209696
597,topi_nn_conv2d,tile_f-tile_y-tile_x-tile_rc-tile_ry-tile_rx-a...,"(512, 512, 3, 3)","(1, 1)","(1, 1, 1, 1)",195140,cuda -model=unknown,resnet-18,"[0.0, 0.0, 0.0, 1.0, 0.0, 2.0, 25.977279663085...",0.0209323
598,topi_nn_conv2d,tile_f-tile_y-tile_x-tile_rc-tile_ry-tile_rx-a...,"(512, 512, 3, 3)","(1, 1)","(1, 1, 1, 1)",138559,cuda -model=unknown,resnet-18,"[0.0, 0.0, 0.0, 1.0, 0.0, 2.0, 0.0, 3.0, 24.78...",0.0209375
